#  1 install and import dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils #drawing helpers
mp_holistic = mp.solutions.holistic #give landmark detection model

#  2. make some detections

In [3]:
cap = cv2.VideoCapture(0) #feed from webcam
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: #loading holistic model
    
    while cap.isOpened(): #looping through each frame coming through webcam
        ret, frame = cap.read() #read feed from webcam, image from webcam inside the variable frame
        
        # Recolor Feed
        #(when we read our frame using opencv by default it will be in bgr format, inorder to work with mediapipe we need rgb frame)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #by setting writeable flags, u prevent copying image data but ur able to use same image for rendering   
        
        # Make Detections
        results = holistic.process(image) #detections from mediapipe model is done using process method and is stored in results
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image) #rendering results to screen

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


#  3. Capture landmark and export to csv

In [4]:
import csv #helps to work with csv file
import os #help us with folder structure
import numpy as np #for working with numpy arrays

In [5]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark) 
num_coords #number of coordinates we have to loop through

501

In [9]:
landmarks = ['class']
for val in range(1, num_coords + 1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)] #each point has four coordinate x,y,z, visibility
#inside of landmarks list of column names (header row we will have inside our csv) class -> happy sad victorous 
landmarks 

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [10]:
#export column name into csv
with open('coords.csv', mode = 'w', newline = '') as f:  
    csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [15]:
#as our model detects different coordinates we're going to write them to our csv, extract landmark and attach to that language
class_name = "Sleepy"
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates
        try:
            # (formating differnt landmark into numpy array, flatten function collapses an array into single dimension [[1,2],[3,4]] into [1,2,3,4])
            # extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            #extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
            #concatenate rows
            row = pose_row + face_row
            
            #append class name to first column
            row.insert(0, class_name)
            
            #export to csv
            with open('coords.csv', mode = 'a', newline = '') as f:
                csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


#  4.Train Custom Model Using Scikit Learn

# read in collected data and process

In [39]:
import pandas as pd #helps to work with tabluar data
from sklearn.model_selection import train_test_split

In [40]:
df = pd.read_csv('coords.csv') 

In [41]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.653656,0.666404,-1.634596,0.995610,0.686018,0.607035,-1.552645,0.994383,0.705129,...,-0.003486,0.0,0.720811,0.618381,0.018386,0.0,0.726578,0.612508,0.019124,0.0
1,Happy,0.653804,0.672055,-1.172112,0.996031,0.686383,0.611359,-1.103787,0.994900,0.705231,...,-0.004215,0.0,0.720180,0.617377,0.017210,0.0,0.725845,0.611059,0.017947,0.0
2,Happy,0.654243,0.675367,-1.137501,0.996399,0.686880,0.613777,-1.073594,0.995352,0.705348,...,-0.004330,0.0,0.719339,0.614556,0.017105,0.0,0.725036,0.608477,0.017796,0.0
3,Happy,0.654294,0.677834,-1.142949,0.996732,0.686889,0.615565,-1.077590,0.995749,0.705326,...,-0.004607,0.0,0.719756,0.613236,0.016614,0.0,0.725328,0.607081,0.017306,0.0
4,Happy,0.654799,0.679276,-1.139459,0.997037,0.687025,0.616517,-1.075193,0.996127,0.705333,...,-0.004422,0.0,0.721199,0.612579,0.017097,0.0,0.726874,0.606469,0.017810,0.0


In [42]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
595,Sleepy,0.550304,0.669122,-1.496503,0.991340,0.586380,0.602186,-1.439169,0.990422,0.603610,...,-0.002648,0.0,0.610380,0.607257,0.020132,0.0,0.615343,0.597106,0.021072,0.0
596,Sleepy,0.549741,0.670866,-1.613244,0.992173,0.584958,0.603585,-1.553818,0.991322,0.601197,...,-0.003792,0.0,0.599896,0.607901,0.017903,0.0,0.604962,0.597475,0.018614,0.0
597,Sleepy,0.548260,0.671673,-1.509156,0.992937,0.582324,0.604436,-1.448300,0.992162,0.600149,...,-0.002169,0.0,0.592465,0.605823,0.019019,0.0,0.597369,0.595660,0.019824,0.0
598,Sleepy,0.542244,0.672875,-1.480042,0.993627,0.574694,0.604515,-1.425733,0.992922,0.594878,...,-0.001818,0.0,0.596825,0.597577,0.019781,0.0,0.601782,0.587934,0.020628,0.0
599,Sleepy,0.533488,0.668002,-1.292364,0.994260,0.569052,0.599201,-1.238297,0.993623,0.591450,...,-0.001841,0.0,0.601997,0.584984,0.019092,0.0,0.607075,0.576796,0.019742,0.0


In [43]:
df[df['class'] == 'Victory']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
345,Victory,0.611744,0.719503,-0.583802,0.999992,0.630628,0.685025,-0.516343,0.999990,0.643153,...,0.000617,0.0,0.655126,0.663204,0.012370,0.0,0.658226,0.659694,0.012807,0.0
346,Victory,0.613830,0.718681,-0.598799,0.999990,0.630934,0.683705,-0.531457,0.999987,0.643286,...,0.001965,0.0,0.659776,0.650907,0.015703,0.0,0.662568,0.647540,0.016270,0.0
347,Victory,0.614122,0.714628,-0.589705,0.999990,0.630980,0.678313,-0.522501,0.999986,0.643289,...,0.000670,0.0,0.660751,0.648653,0.012035,0.0,0.663823,0.645229,0.012449,0.0
348,Victory,0.614689,0.713076,-0.694401,0.999988,0.631524,0.676367,-0.630068,0.999984,0.643789,...,0.001001,0.0,0.661840,0.648330,0.012315,0.0,0.664964,0.644584,0.012769,0.0
349,Victory,0.615097,0.711116,-0.667857,0.999987,0.631967,0.673662,-0.606827,0.999982,0.644171,...,0.001453,0.0,0.662078,0.648669,0.012361,0.0,0.665373,0.644972,0.012755,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,Victory,0.590783,0.671230,-0.859717,0.999781,0.612103,0.621609,-0.774755,0.999599,0.627316,...,-0.001156,0.0,0.640367,0.601442,0.014685,0.0,0.643926,0.597010,0.015293,0.0
450,Victory,0.594967,0.666328,-0.928437,0.999743,0.616347,0.617714,-0.855950,0.999538,0.631433,...,-0.001096,0.0,0.648795,0.598335,0.015896,0.0,0.652394,0.593876,0.016517,0.0
451,Victory,0.597538,0.664597,-0.926206,0.999729,0.618806,0.616493,-0.866499,0.999512,0.633390,...,0.000309,0.0,0.659052,0.599598,0.017870,0.0,0.663060,0.594401,0.018620,0.0
452,Victory,0.598163,0.661850,-0.923880,0.999699,0.623101,0.614944,-0.864731,0.999447,0.638259,...,-0.001506,0.0,0.666198,0.594218,0.016138,0.0,0.670330,0.589575,0.016730,0.0


In [44]:
x = df.drop('class', axis = 1)
y = df['class']

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 1234)

# train machine learning classification model

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [47]:
pipelines = {
    'lr': make_pipeline(StandardScaler(),LogisticRegression()),
    'rc': make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [48]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_models[algo] = model

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [50]:
fit_models['rf'].predict(x_test)

array(['Sleepy', 'Victory', 'Sleepy', 'Happy', 'Happy', 'Sad', 'Sad',
       'Sleepy', 'Happy', 'Happy', 'Victory', 'Happy', 'Sleepy', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Sleepy', 'Victory', 'Happy', 'Sleepy', 'Happy', 'Happy', 'Sleepy',
       'Sad', 'Sleepy', 'Happy', 'Sleepy', 'Sleepy', 'Sleepy', 'Sad',
       'Sad', 'Sad', 'Happy', 'Happy', 'Sleepy', 'Victory', 'Happy',
       'Sad', 'Sleepy', 'Sad', 'Sleepy', 'Sleepy', 'Sleepy', 'Victory',
       'Sad', 'Sleepy', 'Victory', 'Sleepy', 'Sad', 'Sad', 'Sleepy',
       'Happy', 'Happy', 'Sad', 'Sad', 'Victory', 'Victory', 'Victory',
       'Victory', 'Sleepy', 'Sleepy', 'Sleepy', 'Sad', 'Victory', 'Happy',
       'Happy', 'Victory', 'Sad', 'Sad', 'Sad', 'Sad', 'Sad', 'Victory',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Sleepy', 'Victory',
       'Sad', 'Happy', 'Sleepy', 'Victory', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Victory', 'Sad', 'Sad', 'Sleepy', 'Sad',


# evaluate and serialize model

In [51]:
from sklearn.metrics import accuracy_score
import pickle

In [52]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [55]:
fit_models['rf'].predict(x_test)

array(['Sleepy', 'Victory', 'Sleepy', 'Happy', 'Happy', 'Sad', 'Sad',
       'Sleepy', 'Happy', 'Happy', 'Victory', 'Happy', 'Sleepy', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Sleepy', 'Victory', 'Happy', 'Sleepy', 'Happy', 'Happy', 'Sleepy',
       'Sad', 'Sleepy', 'Happy', 'Sleepy', 'Sleepy', 'Sleepy', 'Sad',
       'Sad', 'Sad', 'Happy', 'Happy', 'Sleepy', 'Victory', 'Happy',
       'Sad', 'Sleepy', 'Sad', 'Sleepy', 'Sleepy', 'Sleepy', 'Victory',
       'Sad', 'Sleepy', 'Victory', 'Sleepy', 'Sad', 'Sad', 'Sleepy',
       'Happy', 'Happy', 'Sad', 'Sad', 'Victory', 'Victory', 'Victory',
       'Victory', 'Sleepy', 'Sleepy', 'Sleepy', 'Sad', 'Victory', 'Happy',
       'Happy', 'Victory', 'Sad', 'Sad', 'Sad', 'Sad', 'Sad', 'Victory',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Sleepy', 'Victory',
       'Sad', 'Happy', 'Sleepy', 'Victory', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Victory', 'Sad', 'Sad', 'Sleepy', 'Sad',


In [60]:
y_test

570     Sleepy
380    Victory
457     Sleepy
66       Happy
62       Happy
        ...   
129      Happy
152      Happy
168        Sad
67       Happy
555     Sleepy
Name: class, Length: 180, dtype: object

In [61]:
with open('body_language.pkl' , 'wb') as f:   #write binary
    pickle.dump(fit_models['rf'] , f)

# 4 . Make detection with model

In [33]:
with open('body_language.pkl','rb') as f:     #read binary
    model = pickle.load(f)

In [34]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [64]:
class_name = "Victory"
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        #export coordinates
        try:
            #extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            #extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
            #concatenate rows
            row = pose_row + face_row
            
            #make detections
            x = pd.DataFrame([row])
            body_language_class = model.predict(x)[0]
            body_language_prob = model.predict_proba(x)[0]
            #print(body_language_class, body_language_prob)
            
            #grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                             results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)),
                             [640,480]).astype(int))
            #draws a rectangle onto the image
            cv2.rectangle(image,
                          (coords[0], coords[1] + 5),
                          (coords[0] + len(body_language_class) * 20 , coords[1]-30),
                          (245, 117, 16) , -1)
            #allows to write text onto an image
            cv2.putText(image,body_language_class, coords,
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255) , 2, cv2.LINE_AA)
            
            
            #optional ---->
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature